#  Applied Data Science Capstone: Week 3

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize
import requests
#from bs4 import BeautifulSoup
import html5lib

## Scraping the Table of Canada's Postal Codes

In [2]:
url =  "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" 
page = requests.get(url)
table = pd.read_html(url)

In [3]:
print(f'Total tables: {len(table)}')

Total tables: 3


## We want the first table

In [4]:
table = table[0]

## Because the values we want as our separate columns are listed within each box of the table, we will flatten the table into a vector and then use slicing to pull out the features we want.

In [5]:
postal = table.values.flatten()

In [6]:
postal = pd.DataFrame(postal)

### The Postal Code is contained in the first three positions of the string. We extract it and give it its own column.

In [7]:
postal.columns = ['source']
postal['Postal_Code'] = postal['source'].str[:3]
postal['source'] = postal['source'].str[3:]

### We remove any unassigned Postal Codes.

In [8]:
postal.drop(postal[postal['source'] == "Not assigned"].index, inplace = True) 

### Cleaning up any extra characters to make for easy feature selection

In [9]:
postal["source"] = postal.source.str.replace('[^a-zA-Z]', ' ')
postal.head()

,source,Postal_Code
2,North York Parkwoods,M3A
3,North York Victoria Village,M4A
4,Downtown Toronto Regent Park Harbourfront,M5A
5,North York Lawrence Manor Lawrence Heights,M6A
6,Queen s Park Ontario Provincial Government,M7A


### Splitting the "source" column into multiple columns, using space as a delimiter

In [10]:
new = postal.source.str.split(expand = True)
new.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
2,North,York,Parkwoods,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,North,York,Victoria,Village,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Downtown,Toronto,Regent,Park,Harbourfront,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,North,York,Lawrence,Manor,Lawrence,Heights,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,Queen,s,Park,Ontario,Provincial,Government,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


### Borough is the first column. Then, we concatenate the other columns to form the Neighborhoods column and remove all the "nones"

In [11]:
postal["Borough"] = new.iloc[:,0]

In [12]:
postal["Neighborhoods"] = new.iloc[:, 1:8].apply(lambda row:', '.join(row.values.astype(str)), axis = 1)

In [13]:
postal["Neighborhoods"] = postal.Neighborhoods.str.replace(', None', '')

### Here are our final dataframe and its shape

In [14]:
postal = postal.drop(['source'], axis = 1)
postal.head()

,Postal_Code,Borough,Neighborhoods
2,M3A,North,"York, Parkwoods"
3,M4A,North,"York, Victoria, Village"
4,M5A,Downtown,"Toronto, Regent, Park, Harbourfront"
5,M6A,North,"York, Lawrence, Manor, Lawrence, Heights"
6,M7A,Queen,"s, Park, Ontario, Provincial, Government"


In [15]:
postal.shape

(103, 3)